# Machine Learning Group Project

This notebook contains the code for training our Machine Learning module which classifies the image into 101 categories of food. The dataset that we used for training and testing can be found here. https://pytorch.org/vision/stable/generated/torchvision.datasets.Food101.html#food101.

## Import modules

This section contains code to import all the code that is required for our ML pipeline.

In [1]:
# Import statements

## Import Dataset

This section contains downloading the dataset, changing it into dataloader compatible format and getting a training and validation data with 70-30 split.

### Import dataset from PyTorch

In [2]:
# import dataset from python

### Transform Dataset

Transform the dataset into a format that is compatible with the dataloader function of PyTorch. The PyTorch documentation can be accessed on https://pytorch.org/docs/stable/data.html#torch.utils.data.TensorDataset and https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader.

Make sure to split your dataset into train and test before creating the dataloader

In [3]:
# Transform the dataset in TensorDataset

# Split dataset in to train and test dataset

# Create train and validation dataloader

## Create the model for Image Classification

This section contains the code to either create a model from Scratch or using built-in PyTorch API modules. The basic architecture of the model is x -> (conv2d -> maxpool2d -> activation)* -> flatten -> Linear

In [6]:
# Create a new class that is a child of LightningModule that has all the required methods
# Refer to https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader for a list of require methods

## Training

This section contains the training and logging of the module and the metrics for each step.